# SetFit for Multilabel Text Classification

In this notebook, we'll learn how to do few-shot text classification on a multilabel dataset with SetFit.

## Setup

If you're running this Notebook on Colab or some other cloud platform, you will need to install the `setfit` library. Uncomment the following cell and run it:

In [1]:
# %pip install setfit

To be able to share your model with the community, there are a few more steps to follow.

First, you have to store your authentication token from the Hugging Face Hub (sign up [here](https://huggingface.co/join) if you haven't already!). To do so, execute the following cell and input an [access token](https://huggingface.co/docs/hub/security-tokens) associated with your account:

In [2]:
# from huggingface_hub import notebook_login

# notebook_login()

Then you need to install Git-LFS, which you can do by uncommenting and running following command:

In [3]:
# !apt install git-lfs

Finally, you may need to configue Git on your system by providing details about who you are:

In [4]:
# !git config --global user.email "you@example.com"
# !git config --global user.name "Your Name"

This notebook is designed to work with any multiclass [text classification dataset](https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads) and pretrained [Sentence Transformer](https://huggingface.co/models?library=sentence-transformers&sort=downloads) on the Hub. Change the values below to try a different dataset / model!

In [5]:
from datasets import load_dataset

model_id = "sentence-transformers/paraphrase-mpnet-base-v2"
dataset = load_dataset("ethos", "multilabel")

/Users/maximekuil/Documents/Simplon/fine-tune-classification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/maximekuil/Documents/Simplon/fine-tune-classification/.venv/lib/python3.12/site-packages/datasets/load.py:1491: FutureWarning: The repository for ethos contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ethos
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


## Loading and sampling the dataset

Most datasets on the Hub have many more labeled examples than those one encounters in few-shot settings. To simulate the effect of training on a limited number of examples, let's subsample the training set to have at least 8 labeled examples per feature.

Note that if your dataset has differently formatted labels, you may need to adapt this section.

In [6]:
import numpy as np

features = dataset["train"].column_names
features.remove("text")
features

['violence',
 'directed_vs_generalized',
 'gender',
 'race',
 'national_origin',
 'disability',
 'religion',
 'sexual_orientation']

In [7]:
num_samples = 8
samples = np.concatenate(
    [np.random.choice(np.where(dataset["train"][f])[0], num_samples) for f in features]
)

We encode the emotions in a single `'label'` feature. 

In [8]:
def encode_labels(record):
    return {"labels": [record[feature] for feature in features]}


dataset = dataset.map(encode_labels)

Next, we use the samples we selected as our training set, and the others as our test set (since the ethos dataset does not have a test split on the hub).

Here we have 64 total examples to train with since the `ethos` dataset has 8 classes.

In [9]:
train_dataset = dataset["train"].select(samples)
eval_dataset = dataset["train"].select(
    np.setdiff1d(np.arange(len(dataset["train"])), samples)
)

train_dataset.data

MemoryMappedTable
text: string
violence: int64
directed_vs_generalized: int64
gender: int64
race: int64
national_origin: int64
disability: int64
religion: int64
sexual_orientation: int64
labels: list<item: int64>
  child 0, item: int64
----
text: [["You should know women's sports are a joke","You look like Sloth with deeper Down’s syndrome","You look like Russian and speak like Indian. Both are disgusting go kill yourself","Women deserve to be abused, I guess.","Women are made for making babies and cooking dinner and nothing else!!!",...,"All people are fucked up because of these queer thing because y'all realize how shitty gender roles are and how absurd is to see it in reverse genders roles","Actually THIS WEIRD PEOPLE are HARMFUL! They need medications. His mother should have know he is like this and to not deny it","ABU DA BI ABA DU BA BA HAMUD! See? I’m an arab priest too. This is how they sound. right? They are jokes","a homo? so you fuck your brother instead of your sister?","((

Okay, now we have the dataset, let's load and train a model!

## Fine-tuning the model

To train a SetFit model, the first thing to do is download a pretrained checkpoint from the Hub. We can do so by using the `from_pretrained()` method associated with the `SetFitModel` class.

**Note that the `multi_target_strategy` parameter here signals to both the model and the trainer to expect a multi-labelled dataset.**

In [10]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained(model_id, multi_target_strategy="one-vs-rest")

/Users/maximekuil/Documents/Simplon/fine-tune-classification/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Here, we've downloaded a pretrained Sentence Transformer from the Hub and added a logistic classification head to the create the SetFit model. As indicated in the message, we need to train this model on some labeled examples. We can do so by using the `SetFitTrainer` class as follows:

In [11]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    column_mapping={"text": "text", "labels": "label"},
)

/var/folders/zp/c97k147160ddnklbfc2rk3hr0000gn/T/ipykernel_74208/2403420184.py:4: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.MessageMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.ScalarMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
/Users/maximekuil/Documents/Simplon/fine-tune-classification/.venv/lib/python3.12/site-packages/mlflow/utils/requirements_utils.py:20: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.py

The main arguments to notice in the trainer is the following:

* `loss_class`: The loss function to use for contrastive learning with the Sentence Transformer body
* `num_iterations`: The number of text pairs to generate for contrastive learning
* `column_mapping`: The `SetFitTrainer` expects the inputs to be found in a `text` and `label` column. This mapping automatically formats the training and evaluation datasets for us.

Now that we've created a trainer, we can train it!

In [12]:
trainer.train()

***** Running training *****
  Num unique pairs = 2560
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 160
  0%|          | 0/160 [00:00<?, ?it/s]


KeyboardInterrupt: 

The final step is to compute the model's performance using the `evaluate()` method. The default metric measures 'subset accuracy', which measures the fraction of samples where we predict all 8 labels correctly.

In [ ]:
metrics = trainer.evaluate()
metrics

And once the model is trained, you can push it to the Hub:

In [ ]:
#trainer.push_to_hub(f"setfit-ethos-multilabel-example")

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `your-username/the-name-you-picked` so for instance:

In [ ]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("lewtun/setfit-ethos-multilabel-example")

Run inference. As is usual in toxicity models, it tends to think any mention of topics such as race or gender are negative.

In [ ]:
preds = model(
    [
        "Jewish people often don't eat pork.",
        "Is this lipstick suitable for people with dark skin?"
    ]
)
preds

In [ ]:
# Show predicted labels, requires you to have stored the 'features' somewhere
[[f for f, p in zip(features, ps) if p] for ps in preds]